## Data management

In [163]:
# Importing libraries 
import pandas as pd 
import numpy as np
import tensorflow as tf
from pathlib import Path
import os

In [164]:
tf.__version__

'2.7.0'

In [165]:
place = str (Path(os.getcwd())) + '\\data\\data.csv' # finding database
data = pd.read_csv(place, sep='\t')                  # opening database

In [166]:
df = data.copy()                                    # copying database

In [167]:
df.drop(['dateload'], axis=1, inplace=True)       # dropping dateload column

In [168]:
filt = ~( (df['IE'] == 0 ) | (df['IE'] == 3) | (df['gender'] == 0) )        # Filter for remove invalid rows in column IE and gender

In [169]:
df = df[filt]                           # Applying filter 

In [170]:
# Optimizing DataFrame

intcolumns = df.columns.drop('country')
df[intcolumns] = df[intcolumns].astype(np.int8)
df['country'] = df['country'].astype('category')

In [171]:
# Label Encoding in country column

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['country'] = le.fit_transform(df['country'].values)


In [172]:
# Cutting data in parameters and target

X = df.iloc[:, 0:280].values        # all columns except the last one
y = df.iloc[:,280].values           # getting the last column(target)

In [173]:
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[   5   39  -46 ...    1    2   25]
 [   3   17   21 ...    1    2   19]
 [   5   41  -17 ...    1    1   23]
 ...
 [   3   29  -38 ...    2    2   28]
 [   4   15 -127 ...    2    1   19]
 [   5   57 -122 ...    2    1   25]]
y >>
 [2 1 1 ... 1 1 1]


In [174]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [175]:
# Splitting the data into training and test sets

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [176]:
# Feature Scaling\

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [177]:
print(np.shape(X),'\n',np.shape(X_train),'\n',np.shape(X_test))

(5323, 370) 
 (4258, 370) 
 (1065, 370)


## Neural networks

In [178]:
ann = tf.keras.models.Sequential()   # Initializing the ANN

In [179]:
# Adding the input layer and the hiddens layers
ann.add(tf.keras.layers.Dense(units=3, activation='relu'))
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=3, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [180]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # compiling the ANN

In [189]:
ann.fit(X_train, y_train, batch_size = 16, epochs = 50)

Epoch 1/50
267/267 [==============================] - 1s 3ms/step - loss: -24892052.0000 - accuracy: 0.8166
Epoch 2/50
267/267 [==============================] - 1s 4ms/step - loss: -25962262.0000 - accuracy: 0.8166
Epoch 3/50
267/267 [==============================] - 1s 4ms/step - loss: -27047850.0000 - accuracy: 0.8166
Epoch 4/50
267/267 [==============================] - 1s 4ms/step - loss: -28151288.0000 - accuracy: 0.8166
Epoch 5/50
267/267 [==============================] - 1s 4ms/step - loss: -29270846.0000 - accuracy: 0.8166
Epoch 6/50
267/267 [==============================] - 1s 3ms/step - loss: -30412848.0000 - accuracy: 0.8166
Epoch 7/50
267/267 [==============================] - 1s 3ms/step - loss: -31587808.0000 - accuracy: 0.8166
Epoch 8/50
267/267 [==============================] - 1s 3ms/step - loss: -32798406.0000 - accuracy: 0.8166
Epoch 9/50
267/267 [==============================] - 1s 3ms/step - loss: -34042176.0000 - accuracy: 0.8166
Epoch 10/50
267/267 [=======

In [190]:
y_pred = ann.predict(X_test)

In [194]:
y_pred  = (y_pred > 1)
if y_pred > 1:
    print (y_pred)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [192]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[  0   0   0]
 [877   0   0]
 [188   0   0]]
